In [ ]:
import requests
import pandas as pd
import time
import urllib3
import os


GITHUB_TOKEN = "" 

TARGET_COUNT = 5000  
OUTPUT_FILE = "dataset_human_5k.csv"
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"}

# SSL Hatalarını Yoksay
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


SEARCH_QUERIES = [
    "language:python license:mit stars:5000..30000",  # Çok popülerler
    "language:python license:mit stars:2000..5000",   # Popülerler
    "language:python license:mit stars:1000..2000",   # Ortalar
    "language:python license:mit stars:500..1000",
    "language:python license:mit stars:300..500",
    "language:python license:mit stars:150..300",
    "language:python license:mit stars:80..150",
    "language:python license:mit stars:40..80",       # Az popülerler (burada çok veri var)
    "language:python license:mit stars:10..40"        # Nadirler
]

def get_human_codes():
    print(f"Başlatılıyor...")
    print(f"Veriler '{OUTPUT_FILE}' dosyasına anlık kaydedilecek.\n")
    
    # Eğer dosya varsa üzerine ekleyerek devam et 
    if os.path.exists(OUTPUT_FILE):
        df_existing = pd.read_csv(OUTPUT_FILE)
        collected_data = df_existing.to_dict('records')
        print(f"Önceki oturumdan {len(collected_data)} veri bulundu. Kaldığı yerden devam ediliyor...")
    else:
        collected_data = []

    total_collected = len(collected_data)
    
    
    for query_base in SEARCH_QUERIES:
        if total_collected >= TARGET_COUNT: break
        
        print(f"\nArama Aralığı Değişti: '{query_base}' taranıyor...")
        page = 1
        
        while total_collected < TARGET_COUNT:
            url = f"https://api.github.com/search/repositories?q={query_base}&sort=stars&order=desc&page={page}&per_page=50"
            
            try:
                response = requests.get(url, headers=HEADERS, verify=False)
                
                # Rate Limit (Hız Sınırı) Kontrolü
                if response.status_code == 403 or response.status_code == 429:
                    print("GitHub API limiti. 60 saniye bekle")
                    time.sleep(60)
                    continue
                
                if response.status_code != 200:
                    print(f"Sayfa hatası ({response.status_code}). Geçiliyor.")
                    break 
                
                repos = response.json().get('items', [])
                if not repos:
                    print("Bu aralıktaki repolar bitti. Diğer aralığa geçiliyor.")
                    break

                for repo in repos:
                    if total_collected >= TARGET_COUNT: break
                    
                    # Daha önce bu repoyu aldıysak tekrar alma (Tekillik kontrolü)
                    if any(d['source'] == repo['html_url'] for d in collected_data):
                        continue

                    print(f"[{total_collected}/{TARGET_COUNT}] {repo['name']}...", end=" ")
                    
                    # İçeriğe Gir
                    default_branch = repo['default_branch']
                    tree_url = f"{repo['url']}/git/trees/{default_branch}?recursive=1"
                    
                    try:
                        tree_resp = requests.get(tree_url, headers=HEADERS, verify=False)
                        if tree_resp.status_code != 200: 
                            print("-> (Gizli/Erişim Yok)")
                            continue
                            
                        files = tree_resp.json().get('tree', [])
                        if tree_resp.json().get('truncated'):
                            # Çok büyük repolarda sadece üst dosyalara bakmak için basit filtre
                            pass 

                        file_added_count = 0
                        
                        # Her repodan en fazla 3 dosya alalım (Çeşitlilik için)
                        for item in files:
                            if file_added_count >= 3: break
                            
                            if item['path'].endswith('.py') and item['type'] == 'blob':
                                raw_url = f"https://raw.githubusercontent.com/{repo['full_name']}/{default_branch}/{item['path']}"
                                code_resp = requests.get(raw_url, verify=False)
                                
                                if code_resp.status_code == 200:
                                    code_text = code_resp.text
                                    
                                    # Kod kalitesi: Çok kısa veya devasa dosyaları alma
                                    if 150 < len(code_text) < 20000:
                                        collected_data.append({
                                            "code": code_text,
                                            "label": "human",
                                            "source": repo['html_url'],
                                            "license": "MIT",
                                            "filename": item['path'].split('/')[-1]
                                        })
                                        total_collected += 1
                                        file_added_count += 1
                                        print("Başarılı", end="")
                        
                        if file_added_count > 0:
                            print(f" -> Kaydedildi.")
                            # HER 10 VERİDE BİR DOSYAYI GÜNCELLE
                            if total_collected % 10 == 0:
                                pd.DataFrame(collected_data).to_csv(OUTPUT_FILE, index=False)
                        else:
                            print("-> (Uygun .py yok)")
                            
                    except Exception as e:
                        print(f"Hata: {e}")
                        continue
                
                page += 1
                
                if page > 30: 
                    break
                    
            except Exception as e:
                print(f"Genel Bağlantı Hatası: {e}")
                time.sleep(10)

    # Final Kayıt
    df_final = pd.DataFrame(collected_data)
    df_final.to_csv(OUTPUT_FILE, index=False)
    print(f"\nGTAMAMLANDI! Toplam {len(df_final)} adet veri '{OUTPUT_FILE}' dosyasına kaydedildi.")
    return df_final

# Başlat
get_human_codes()

Başlatılıyor...
Veriler 'dataset_human_5k.csv' dosyasına anlık kaydedilecek.

Önceki oturumdan 5001 veri bulundu. Kaldığı yerden devam ediliyor...

GTAMAMLANDI! Toplam 5001 adet veri 'dataset_human_5k.csv' dosyasına kaydedildi.


,code,label,source,license,filename
0,import numpy as np\n\n# Collection of activati...,human,https://github.com/eriklindernoren/ML-From-Scr...,MIT,activation_functions.py
1,from __future__ import division\nimport numpy ...,human,https://github.com/eriklindernoren/ML-From-Scr...,MIT,loss_functions.py
2,"from __future__ import print_function, divisio...",human,https://github.com/eriklindernoren/ML-From-Scr...,MIT,neural_network.py
3,# Copyright (c) 2024 Microsoft Corporation.\n#...,human,https://github.com/microsoft/graphrag,MIT,__main__.py
4,# Copyright (c) 2024 Microsoft Corporation.\n#...,human,https://github.com/microsoft/graphrag,MIT,__init__.py
...,...,...,...,...,...
4996,"# -*- coding: utf-8 -*-\n\n""""""\ngmail.exceptio...",human,https://github.com/charlierguo/gmail,MIT,exceptions.py
4997,import re\nimport imaplib\n\nfrom mailbox impo...,human,https://github.com/charlierguo/gmail,MIT,gmail.py
4998,import warnings\n\n# 关闭警告\nwarnings.filterwarn...,human,https://github.com/Aabyss-Team/ARL,MIT,__init__.py
4999,import signal\nimport time\nfrom bson import O...,human,https://github.com/Aabyss-Team/ARL,MIT,celerytask.py


In [ ]:
import requests


MY_TOKEN = "" 

headers = {"Authorization": f"token {MY_TOKEN}"}

print("Bağlantı testi yapılıyor...")
try:
    # Sadece kimlik doğrulaması kontrolü
    response = requests.get("https://api.github.com/user", headers=headers)
    
    if response.status_code == 200:
        username = response.json().get('login')
        print("\n✅ BAŞARILI! Token çalışıyor.")
        print(f"Giriş yapılan kullanıcı: {username}")
        print("Şimdi diğer kod hücresine geçip veri çekmeye başlayabilirsiniz.")
    elif response.status_code == 401:
        print("\n❌ HATA: 401 Unauthorized (Yetkisiz)")
        print("Sebep: Token yanlış kopyalandı veya süresi dolmuş.")
        print("Kontrol: Token 'ghp_' ile mi başlıyor? Tırnak içinde boşluk var mı?")
    else:
        print(f"\n⚠️ Başka bir hata oluştu. Kod: {response.status_code}")
        print(response.text)

except Exception as e:
    print(f"Bağlantı hatası: {e}")

Bağlantı testi yapılıyor...

✅ BAŞARILI! Token çalışıyor.
Giriş yapılan kullanıcı: talhagurun
Şimdi diğer kod hücresine geçip veri çekmeye başlayabilirsiniz.


In [3]:
import pandas as pd

# Dosya adını kontrol et (kodda verdiğin isimle aynı olmalı)
df = pd.read_csv("dataset_human_5k.csv")

print(f"Şu an toplanan veri sayısı: {len(df)}")
df.tail() # Son eklenen 5 veriyi gösterir

Şu an toplanan veri sayısı: 5001


,code,label,source,license,filename
4996,"# -*- coding: utf-8 -*-\n\n""""""\ngmail.exceptio...",human,https://github.com/charlierguo/gmail,MIT,exceptions.py
4997,import re\nimport imaplib\n\nfrom mailbox impo...,human,https://github.com/charlierguo/gmail,MIT,gmail.py
4998,import warnings\n\n# 关闭警告\nwarnings.filterwarn...,human,https://github.com/Aabyss-Team/ARL,MIT,__init__.py
4999,import signal\nimport time\nfrom bson import O...,human,https://github.com/Aabyss-Team/ARL,MIT,celerytask.py
5000,import os\nimport yaml\nimport sys\n\nbasedir ...,human,https://github.com/Aabyss-Team/ARL,MIT,config.py


In [ ]:
import google.generativeai as genai
import pandas as pd
import time
import os
from tqdm import tqdm

# --- AYARLAR ---
GOOGLE_API_KEY = "" 
INPUT_FILE = "dataset_human_5k.csv"   
OUTPUT_FILE = "dataset_ai_5k.csv"     


genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-2.5-flash')

def generate_ai_dataset():
    # 1. İnsan Verisini Oku
    if not os.path.exists(INPUT_FILE):
        print(f"HATA: '{INPUT_FILE}' dosyası bulunamadı. Önce insan verisi çekme işlemini bitirin.")
        return

    df_human = pd.read_csv(INPUT_FILE)
    human_filenames = df_human['filename'].tolist() # Dosya isimlerini al
    
    print(f"İnsan veri seti yüklendi. Hedef: {len(human_filenames)} adet AI kodu üretmek.")

    # 2. Kaldığı Yerden Devam Etme (Resume)
    if os.path.exists(OUTPUT_FILE):
        df_existing = pd.read_csv(OUTPUT_FILE)
        ai_data = df_existing.to_dict('records')
        print(f"Önceki oturumdan {len(ai_data)} veri bulundu. Kaldığı yerden devam ediliyor...")
    else:
        ai_data = []

    start_index = len(ai_data)
    
    # 3. Üretim Döngüsü
    print("AI Kod Üretimi Başlıyor (Gemini 1.5 Flash)...")
    
    # İlerleme çubuğu ile döngü
    for i in tqdm(range(start_index, len(human_filenames)), desc="Üretiliyor"):
        target_filename = human_filenames[i]
        
        # Prompt: Dosya ismine göre kod istiyoruz
        prompt = f"""
        Act as a Python developer. Write a complete, functional Python script for a file named '{target_filename}'.
        RULES:
        1. Provide ONLY the raw code.
        2. Do NOT write explanations, comments intro, or markdown backticks (```).
        3. The code should be roughly 20-100 lines long.
        """
        
        try:
            # API İsteği
            response = model.generate_content(prompt)
            
            
            code_content = response.text.replace("```python", "").replace("```", "").strip()
            
            if len(code_content) > 50:
                ai_data.append({
                    "code": code_content,
                    "label": "ai",
                    "source": "Gemini-2.5-Flash",
                    "license": "Generated",
                    "filename": target_filename
                })
            
            # HER 10 VERİDE BİR KAYDET 
            if len(ai_data) % 10 == 0:
                pd.DataFrame(ai_data).to_csv(OUTPUT_FILE, index=False)
            
            # Hız sınırı (Rate Limit) için kısa bekleme
            
            time.sleep(1.5)

        except Exception as e:
            # Eğer limit hatası (429) alırsa uzun bekle
            if "429" in str(e):
                print(f"\nAPI Kotası Doldu (429). 60 saniye soğutuluyor...")
                time.sleep(60)
            else:
                print(f"\nHata ({target_filename}): {e}")
                time.sleep(2)

    # BİTİŞ
    df_final = pd.DataFrame(ai_data)
    df_final.to_csv(OUTPUT_FILE, index=False)
    print(f"\nTEBRİKLER! Toplam {len(df_final)} AI kodu üretildi ve '{OUTPUT_FILE}' dosyasına kaydedildi.")

# Çalıştır
generate_ai_dataset()

İnsan veri seti yüklendi. Hedef: 5001 adet AI kodu üretmek.
Önceki oturumdan 3870 veri bulundu. Kaldığı yerden devam ediliyor...
AI Kod Üretimi Başlıyor (Gemini 1.5 Flash)...


Üretiliyor: 100%|████████████████████████████████████████████████████████████████| 1131/1131 [4:19:11<00:00, 13.75s/it]



TEBRİKLER! Toplam 5001 AI kodu üretildi ve 'dataset_ai_5k.csv' dosyasına kaydedildi.


In [6]:
!python.exe -m pip install --upgrade pip

  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2


In [ ]:
import google.generativeai as genai

# API Key'inizi buraya tekrar yazın
genai.configure(api_key="")

print("Kullanılabilir Modeller:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(f"- {m.name}")

Kullanılabilir Modeller:
- models/gemini-2.5-flash
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/gemma-3-1b-it
- models/gemma-3-4b-it
- models/gemma-3-12b-it
- models/gemma-3-27b-it
- models/gemma-3n-e4b-it
- models/gemma-3n-e2b-it
- models/gemini-flash-latest
- models/gemini-flash-lite-latest
- models/gemini-pro-latest
- models/gemini-2.5-flash-lite
- models/gemini-2.5-flash-image-preview
- models/gemini-2.5-flash-image
- models/gemini-2.5-flash-preview-09-2025
- models/gemini-2.5-flash-lite-preview-09-2025
- models/gemini-3-pro-preview
- models/gemini-3-pro-ima

In [2]:
import pandas as pd

try:
    df_human = pd.read_csv("dataset_human_5k.csv")
    df_ai = pd.read_csv("dataset_ai_5k.csv")
    print(f"İnsan verisi sayısı {len(df_human)}")
    print(f"AI verisi sayısı {len(df_ai)}")
except FileNotFoundError:
    print("Hata: Dosya/dosyalar bulunamadı")


df_human['label'] = 0
df_ai['label'] = 1

df_human['class_name'] = 'Human'
df_ai['class_name'] = 'AI'

df_final = pd.concat([df_human , df_ai] , ignore_index=True)

df_final = df_final.sample(frac = 1 , random_state=42).reset_index(drop = True)

print("\nBirleştirme Sonrası Durum : ")
print(df_final['class_name'].value_counts())

df_final.to_csv("final_dataset_merged.csv" , index = False)
print("Başarılı!! Dosya oluşturuldu")

df_final.head()

İnsan verisi sayısı 5001
AI verisi sayısı 5001

Birleştirme Sonrası Durum : 
class_name
AI       5001
Human    5001
Name: count, dtype: int64
Başarılı!! Dosya oluşturuldu


,code,label,source,license,filename,class_name
0,import argparse\n\ndef handle_greet(args):\n ...,1,Gemini-2.5-Flash,Generated,cli.py,AI
1,# Take a screenshot of the screen\nimport json...,0,https://github.com/Blaizzy/mlx-vlm,MIT,autonomous_gui_agent.py,Human
2,import logging\nimport os\nimport json\nimport...,0,https://github.com/scottrogowski/code2flow,MIT,javascript.py,Human
3,from .utils import md\n\n\ndef test_chomp():\n...,0,https://github.com/matthewwithanm/python-markd...,MIT,test_advanced.py,Human
4,"""minesweeper""\nfrom UserDict import UserDict\n...",0,https://github.com/ActiveState/code,MIT,recipe-101853.py,Human
